In [9]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2 as cv

In [48]:
data_generator=tf.keras.preprocessing.image.ImageDataGenerator(
rescale=1./255,
validation_split=0.2)

In [4]:
IMAGE_SIZE=224
BATCH_SIZE=5

In [7]:
folders=sorted(os.listdir('/home/rishi/Desktop/MachineLearning/Dataset/train'))
print(folders)

['aishwarya_rai', 'angelina_jolie', 'arnold_schwarzenegger', 'bhuvan_bam', 'brad_pitt', 'cristiano_ronaldo', 'hardik_pandya', 'lionel_messi', 'manoj_bajpayee', 'ms_dhoni', 'pankaj_tripathi', 'rohit_sharma', 'sachin_tendulkar', 'scarlett_johansson', 'virat_kohli']


In [12]:
image_array=[]
label_array=[]
path='/home/rishi/Desktop/MachineLearning/Dataset/train/'
for i in range(len(folders)):
    file_sub=os.listdir(path+folders[i])
    
    for k in tqdm(range(len(file_sub))):
        try:
            img=cv.imread(path+folders[i]+'/'+file_sub[k])
            img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
            img=cv.resize(img,(160,160))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 248.48it/s]


In [18]:
def generate_train_image_pairs(images_dataset,labels_dataset):
    unique_labels=np.unique(labels_dataset)
    label_wise_indices=dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                 [index for index,curr_label in enumerate(labels_dataset)
                                  if label==curr_label])
    pair_images=[]
    pair_labels=[]
    for index,image in enumerate(images_dataset):
        pos_indices=label_wise_indices.get(labels_dataset[index])
        pos_image=images_dataset[np.random.choice(pos_indices)]
        pair_images.append((image,pos_image))
        pair_labels.append(1)
        
        neg_indices=[i for i in range(len(labels_dataset)) if i not in pos_indices]
        neg_image=images_dataset[np.random.choice(neg_indices)]
        pair_images.append((image,neg_image))
        pair_labels.append(0)
    return np.array(pair_images),np.array(pair_labels)

In [19]:
images_pair,labels_pair=generate_train_image_pairs(image_array,label_array)

In [39]:
print(images_pair.shape)

(600, 2, 160, 160, 3)


In [40]:
print(labels_pair.shape)

(600,)


In [57]:
IMG_SIZE=224
BATCH_SIZE=5
IMG_SHAPE=(224,224,3)

In [51]:
train_generator=data_generator.flow_from_directory(
path,
target_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE,
subset='validation')


val_generator=data_generator.flow_from_directory(
path,
target_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE,
subset='validation')

for image_batch ,label_batch in train_generator:
    break

Found 60 images belonging to 15 classes.
Found 60 images belonging to 15 classes.


In [58]:
base_model=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

9406464/9406464 [==============================] - 6s 1us/step


In [59]:
base_model.trainable=False

In [63]:
model=tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(15,activation='softmax')
])

In [64]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [66]:
history=model.fit(train_generator,epochs=100,validation_data=val_generator)

Epoch 1/100
12/12 [==============================] - 3s 284ms/step - loss: 1.1327 - accuracy: 0.6167 - val_loss: 1.0079 - val_accuracy: 0.6333
Epoch 2/100
12/12 [==============================] - 3s 228ms/step - loss: 1.0052 - accuracy: 0.6333 - val_loss: 0.9211 - val_accuracy: 0.6333
Epoch 3/100
12/12 [==============================] - 3s 227ms/step - loss: 0.8956 - accuracy: 0.6667 - val_loss: 0.8414 - val_accuracy: 0.6667
Epoch 4/100
12/12 [==============================] - 3s 227ms/step - loss: 0.8364 - accuracy: 0.7000 - val_loss: 0.7678 - val_accuracy: 0.7000
Epoch 5/100
12/12 [==============================] - 3s 224ms/step - loss: 0.7774 - accuracy: 0.7167 - val_loss: 0.6892 - val_accuracy: 0.7500
Epoch 6/100
12/12 [==============================] - 3s 220ms/step - loss: 0.6845 - accuracy: 0.7333 - val_loss: 0.6227 - val_accuracy: 0.7333
Epoch 7/100
12/12 [==============================] - 3s 222ms/step - loss: 0.5924 - accuracy: 0.7667 - val_loss: 0.5427 - val_accuracy: 0.8000

Epoch 58/100
12/12 [==============================] - 3s 255ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 59/100
12/12 [==============================] - 3s 232ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 60/100
12/12 [==============================] - 3s 228ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 61/100
12/12 [==============================] - 3s 225ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 62/100
12/12 [==============================] - 3s 244ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 63/100
12/12 [==============================] - 3s 226ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 64/100
12/12 [==============================] - 3s 228ms/step - loss: 0.0179 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy:

In [68]:
img=cv.imread('/home/rishi/Desktop/MachineLearning/Dataset/test/angelina_jolie')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d_4 (Conv2D)           (None, 5, 5, 32)          368672    
                                                                 
 dropout_4 (Dropout)         (None, 5, 5, 32)          0         
                                                                 
 global_average_pooling2d_4   (None, 32)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 15)                495       
                                                                 
Total params: 2,627,151
Trainable params: 369,167
Non-

In [72]:
path1='/home/rishi/Desktop/MachineLearning/Dataset/test'

In [73]:
test_generator=data_generator.flow_from_directory(
path1,
target_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE,
subset='validation')


valT_generator=data_generator.flow_from_directory(
path,
target_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE,
subset='validation')

for image_batch ,label_batch in test_generator:
    break

Found 0 images belonging to 16 classes.
Found 60 images belonging to 15 classes.
